# 输入输出

## 文件

### 读取全部内容

In [1]:
with open('/etc/hosts', 'r') as f:
    print(f.read())

##
# Host Database
#
# localhost is used to configure the loopback interface
# when the system is booting.  Do not change this entry.
##
127.0.0.1	localhost
255.255.255.255	broadcasthost
::1             localhost



### 读取多个字节

In [2]:
with open('/etc/hosts', 'r') as f:
    print(f.read(32))

##
# Host Database
#
# localhost


### 读取一行

In [3]:
with open('/etc/hosts', 'r') as f:
    print(f.readline())

##



### 读取所有行并返回列表

In [4]:
with open('/etc/hosts', 'r') as f:
    print(f.readlines())

['##\n', '# Host Database\n', '#\n', '# localhost is used to configure the loopback interface\n', '# when the system is booting.  Do not change this entry.\n', '##\n', '127.0.0.1\tlocalhost\n', '255.255.255.255\tbroadcasthost\n', '::1             localhost\n']


### 将文件描述符包装成文件对象

In [5]:
from socket import socket, AF_INET, SOCK_STREAM

def echo_client(client_sock, addr):
    print("Got connection from", addr)

    # Make text-mode file wrappers for socket reading/writing
    client_in = open(client_sock.fileno(), 'rt', encoding='latin-1', closefd=False)
    client_out = open(client_sock.fileno(), 'wt', encoding='latin-1', closefd=False)

    # Echo lines back to the client using file I/O
    for line in client_in:
        client_out.write(line)
        client_out.flush()
    client_sock.close()

def echo_server(address):
    sock = socket(AF_INET, SOCK_STREAM)
    sock.bind(address)
    sock.listen(1)
    while True:
        client, addr = sock.accept()
        echo_client(client, addr)

print('Echo serving running on localhost:25000')
echo_server(('', 25000))

Echo serving running on localhost:25000


### 改变已打开文件的编码方式

In [ ]:
# Example of adding a text encoding to existing file-like object

import urllib.request
import io

u = urllib.request.urlopen('http://www.python.org')
f = io.TextIOWrapper(u, encoding='utf-8')
text = f.read()

print(text)

## 内存字符串

In [6]:
from io import StringIO
f = StringIO()
f.write('hello')
f.write(' ')
f.write('world!')
f.getvalue()

'hello world!'

In [7]:
f = StringIO('Hello!\nHi!\nGoodbye!')
while True:
    s = f.readline()
    if s == '':  # eof
        break
    print(s, end='')

Hello!
Hi!
Goodbye!

## 内存比特流

In [8]:
from io import BytesIO
f = BytesIO()
f.write('中文'.encode('utf-8'))
f.getvalue()

b'\xe4\xb8\xad\xe6\x96\x87'

In [9]:
f = BytesIO(b'\xe4\xb8\xad\xe6\x96\x87')
f.read().decode('utf-8')

'中文'

## 读写 json

In [11]:
# Some advanced JSON examples involving ordered dicts and classes
import json

# Some JSON encoded text
s = '{"name": "ACME", "shares": 50, "price": 490.1}'

# (a) Turning JSON into an OrderedDict

from collections import OrderedDict
data = json.loads(s, object_pairs_hook=OrderedDict)
print(data)

# (b) Using JSON to populate an instance

class JSONObject:
    def __init__(self, d):
        self.__dict__ = d

data = json.loads(s, object_hook=JSONObject)
print(data.name)
print(data.shares)
print(data.price)

# (c) Encoding instances

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

def serialize_instance(obj):
    d = { '__classname__' : type(obj).__name__ }
    d.update(vars(obj))
    return d

p = Point(3,4)
s = json.dumps(p, default=serialize_instance)
print(s)

# (d) Decoding instances
classes = {
    'Point' : Point
}

def unserialize_object(d):
    clsname = d.pop('__classname__', None)
    if clsname:
        cls = classes[clsname]
        obj = cls.__new__(cls)
        for key, value in d.items():
            setattr(obj, key, value)
        return obj
    else:
        return d

a = json.loads(s, object_hook=unserialize_object)
print(a)
print(a.x)
print(a.y)

OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])
ACME
50
490.1
{"y": 4, "__classname__": "Point", "x": 3}
3
4
